# 📓 4. Run Queries
## Query Performance Benchmarking for TC.11.x–TC.13.x

### 🔗 Ensure `BenchmarkLakehouse` is connected as a data source before running.

Assumes synthetic data, initial load, and updates have been completed.
All queries are run against the target table in BenchmarkLakehouse and BenchmarkWarehouse.
Join queries are excluded (single table per location).


In [ ]:
import time
from pyspark.sql.functions import col

In [ ]:
# Paths and table names
target_lakehouse = "BenchmarkLakehouse"
target_warehouse = "BenchmarkWarehouse"
row_count = 10000  # Update as appropriate

delta_tables = {
    "refresh": "delta_refresh_load",
    "compare": "delta_compare_load",
    "increment": "delta_increment_load"
}
warehouse_tables = {
    "refresh": "wh_table_refresh_load",
    "compare": "wh_table_compare_load",
    "increment": "wh_table_increment_load"
}

# Choose table to query (usually 'increment' for event log)
lakehouse_table = f"{target_lakehouse}.{delta_tables['increment']}"
warehouse_table = f"{target_warehouse}.dbo.{warehouse_tables['increment']}"

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType, FloatType
from datetime import datetime

# NOTE: storage_size_mb and cu_used are INTEGER row-count proxies in this pipeline.
metrics_schema = StructType([
    StructField("test_case_id", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("format", StringType(), True),
    StructField("location", StringType(), True),
    StructField("rows", IntegerType(), True),
    StructField("update_strategy", StringType(), True),
    StructField("ingest_time_s", FloatType(), True),
    StructField("spinup_time_s", FloatType(), True),
    StructField("storage_size_mb", IntegerType(), True),  # row-count proxy
    StructField("query_type", StringType(), True),
    StructField("query_time_s", FloatType(), True),
    StructField("cu_used", IntegerType(), True),          # rows-processed proxy
    StructField("notes", StringType(), True)
])

In [ ]:
# Read Delta table from Lakehouse
df_lakehouse = spark.read.table(lakehouse_table)

## Query Types
- **Filter Query**: Select rows with a specific category value.
- **Aggregate Query**: Group by category, aggregate numeric columns.
- **Batch Query**: Select update events in a specific time window.
- **Top-N Query**: Retrieve top N rows by a numeric column.

*(Join queries are excluded—only one table in each target location.)*

In [ ]:
# Define query functions
def run_filter_query(df):
    # Example: Filter by category
    return df.filter(col("cat_1") == "A").count()

def run_aggregate_query(df):
    # Example: Group by category, aggregate numerics
    return df.groupBy("cat_1").agg({"num_1": "avg", "num_2": "max"}).count()

def run_batch_query(df):
    # Example: Select events by update_type and time window
    return df.filter((col("update_type") == "update") & (col("ts_1") > "2025-01-01")).count()

def run_topn_query(df, n=10):
    # Example: Top-N by numeric value
    return df.orderBy(col("num_1").desc()).limit(n).count()

In [ ]:
# Performance logging utility
def log_query_perf(query_func, df, description):
    start = time.time()
    result = query_func(df)
    elapsed = time.time() - start
    print(f"{description}: {elapsed:.3f}s (Rows: {result})")
    return {"query": description, "rows": result, "time_s": elapsed}

In [ ]:
# Run queries on Lakehouse Delta table
lakehouse_metrics = []
lakehouse_metrics.append(log_query_perf(run_filter_query, df_lakehouse, "Lakehouse Filter cat_1 == 'A'"))
lakehouse_metrics.append(log_query_perf(run_aggregate_query, df_lakehouse, "Lakehouse Aggregate by cat_1"))
lakehouse_metrics.append(log_query_perf(run_batch_query, df_lakehouse, "Lakehouse Batch update_type == update, ts_1 > '2025-01-01'"))
lakehouse_metrics.append(log_query_perf(run_topn_query, df_lakehouse, "Lakehouse Top 10 num_1"))

In [ ]:
# Read Warehouse table (for Spark SQL endpoint, not as DataFrame)
from com.microsoft.spark.fabric import Constants
df_warehouse = spark.read.synapsesql(warehouse_table)

In [ ]:
# Run queries on Warehouse table
warehouse_metrics = []
warehouse_metrics.append(log_query_perf(run_filter_query, df_warehouse, "Warehouse Filter cat_1 == 'A'"))
warehouse_metrics.append(log_query_perf(run_aggregate_query, df_warehouse, "Warehouse Aggregate by cat_1"))
warehouse_metrics.append(log_query_perf(run_batch_query, df_warehouse, "Warehouse Batch update_type == update, ts_1 > '2025-01-01'"))
warehouse_metrics.append(log_query_perf(run_topn_query, df_warehouse, "Warehouse Top 10 num_1"))

In [ ]:
# Display metrics as table
import pandas as pd

all_metrics = lakehouse_metrics + warehouse_metrics
metrics_df = pd.DataFrame(all_metrics)
display(metrics_df)

# Print completion message
print("Query performance benchmarking complete. Metrics above can be visualized in the next step.")

In [ ]:
# Log metrics to metrics table in BenchmarkLakehouse
def log_query_to_metrics(test_case_id, format, location, rows, query_type, query_time_s, notes=""):
    # Use None for fields that are not applicable. storage_size_mb and cu_used are INTEGER row-count proxies elsewhere
    metrics_row = [(
        test_case_id,
        datetime.now(),
        format,
        location,
        rows,
        "",                  # update_strategy (N/A for queries)
        None,                 # ingest_time_s (N/A for queries)
        None,                 # spinup_time_s (N/A)
        None,                 # storage_size_mb (N/A for queries)
        query_type,
        float(query_time_s),
        None,                 # cu_used (N/A for queries)
        notes
    )]
    spark.createDataFrame(metrics_row, schema=metrics_schema).write.mode('append').saveAsTable(f'{target_lakehouse}.metrics')

In [ ]:
# Log all Lakehouse query metrics
for metric in lakehouse_metrics:
    log_query_to_metrics(
        test_case_id="TC.11.x",
        format="Delta",
        location="Tables",
        rows=metric['rows'],
        query_type=metric['query'],
        query_time_s=metric['time_s'],
        notes="Lakehouse query performance"
    )

# Log all Warehouse query metrics
for metric in warehouse_metrics:
    log_query_to_metrics(
        test_case_id="TC.12.x",
        format="Warehouse",
        location="Tables",
        rows=metric['rows'],
        query_type=metric['query'],
        query_time_s=metric['time_s'],
        notes="Warehouse query performance"
    )

# 📊 Visualize Metrics
### 🔗 Ensure `BenchmarkLakehouse` is connected as a data source before running.

This section appends the new visualizations to the notebook. It reads the metrics table and creates a 5×6 matrix of small charts comparing formats and strategies, plus the standalone visuals for[...]
Notes:
- The notebook expects storage_size_mb and cu_used to be integer row-count proxies (per prior changes).
- The visualizations below prefer recorded metrics from the lakehouse metrics table and fall back to a local metrics.csv for development.


In [ ]:
# Load metrics table from Lakehouse (or local CSV for dev) and normalize proxy fields
import pandas as pd
import numpy as np
try:
    metrics_df = spark.read.table("BenchmarkLakehouse.metrics").toPandas()
except Exception:
    metrics_df = pd.read_csv("metrics.csv")

# Normalize numeric proxy fields (storage_size_mb and cu_used may be stored as floats in the metrics table)
metrics_df['storage_size_mb'] = pd.to_numeric(metrics_df.get('storage_size_mb', pd.Series(np.nan)), errors='coerce')
metrics_df['cu_used'] = pd.to_numeric(metrics_df.get('cu_used', pd.Series(np.nan)), errors='coerce')

# Create integer proxy columns for display and plotting (None for missing)
def to_int_proxy(x):
    if pd.isna(x):
        return None
    try:
        return int(x)
    except Exception:
        try:
            return int(float(x))
        except Exception:
            return None

metrics_df['storage_rows_proxy'] = metrics_df['storage_size_mb'].apply(to_int_proxy)
metrics_df['cu_used_int'] = metrics_df['cu_used'].apply(to_int_proxy)

metrics_df.head()


In [ ]:
# Calculate storage_size_mb if missing (best effort)
import math
import pandas as pd

def calculate_storage_for_table(table_path):
    try:
        import mssparkutils
        files = mssparkutils.fs.ls(table_path)
        size_mb = sum(f.size for f in files) / (1024 * 1024)
        return size_mb
    except Exception:
        return float('nan')

# Identify unique targets and fill missing storage_size_mb when possible
for idx, row in metrics_df.iterrows():
    if ('storage_size_mb' in row and (pd.isna(row['storage_size_mb']) or math.isnan(row['storage_size_mb']))) and row['update_strategy']:
        # crude extraction: use test_case_id to guess table name (customize if needed)
        tc = row['test_case_id']
        table_map = {
            'TC.01.x': '/lakehouse/BenchmarkLakehouse/Tables/delta_refresh_load',
            'TC.02.x': '/lakehouse/BenchmarkLakehouse/Tables/wh_table_refresh_load',
            'TC.03.x': '/lakehouse/BenchmarkLakehouse/Tables/delta_refresh_load',
            'TC.04.x': '/lakehouse/BenchmarkLakehouse/Tables/wh_table_refresh_load',
            'TC.05.x': '/lakehouse/BenchmarkLakehouse/Tables/delta_compare_load',
            'TC.06.x': '/lakehouse/BenchmarkLakehouse/Tables/wh_table_compare_load',
            'TC.07.x': '/lakehouse/BenchmarkLakehouse/Tables/delta_increment_load',
            'TC.08.x': '/lakehouse/BenchmarkLakehouse/Tables/wh_table_increment_load',
        }
        table_path = table_map.get(tc, None)
        if table_path:
            metrics_df.at[idx, 'storage_size_mb'] = calculate_storage_for_table(table_path)
            # refresh proxy column when we populate storage_size_mb
            try:
                metrics_df.at[idx, 'storage_rows_proxy'] = int(metrics_df.at[idx, 'storage_size_mb'])
            except Exception:
                metrics_df.at[idx, 'storage_rows_proxy'] = None


In [ ]:
# Initial ingestion performance
import matplotlib.pyplot as plt
ingest_df = metrics_df[metrics_df['update_strategy'] == 'Full Refresh']
plt.figure(figsize=(8,4))
plt.bar(ingest_df['format'], ingest_df['ingest_time_s'], color=['skyblue', 'orange'])
plt.title("Initial Ingestion Time by Format")
plt.xlabel("Format")
plt.ylabel("Ingestion Time (s)")
plt.show()


In [ ]:
# Update performance comparison
update_df = metrics_df[metrics_df['update_strategy'].isin(['Full Compare', 'Incremental'])]
plt.figure(figsize=(8,4))
for strategy in update_df['update_strategy'].unique():
    strat_df = update_df[update_df['update_strategy'] == strategy]
    plt.bar(strat_df['format'] + " " + strat_df['update_strategy'], strat_df['ingest_time_s'], label=strategy)
plt.title("Update Time by Strategy and Format")
plt.xlabel("Strategy")
plt.ylabel("Update Time (s)")
plt.legend()
plt.show()


In [ ]:
# Query performance comparison
query_df = metrics_df[metrics_df['query_type'].notna()]
plt.figure(figsize=(10,5))
plt.bar(query_df['query_type'], query_df['query_time_s'], color='seagreen')
plt.xticks(rotation=45)
plt.title("Query Performance Comparison")
plt.xlabel("Query Type")
plt.ylabel("Query Time (s)")
plt.tight_layout()
plt.show()


In [ ]:
# Storage cost comparison (use integer proxy column created above)
storage_df = metrics_df.dropna(subset=['storage_rows_proxy'])
storage_summary = storage_df.groupby(['format', 'update_strategy'])['storage_rows_proxy'].mean().reset_index()
plt.figure(figsize=(10,5))
plt.bar(storage_summary['format'] + " " + storage_summary['update_strategy'], storage_summary['storage_rows_proxy'], color='orchid')
plt.title("Storage Size by Target Table")
plt.xlabel("Target Table")
plt.ylabel("Storage Size (rows) — row-count proxy")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Show all metrics as interactive table
import seaborn as sns
sns.set(style="whitegrid")
display(metrics_df)


In [ ]:
print("Visualization complete. Review charts above for performance and storage comparisons across ingestion, update, and query activities.")

# Visualize Metrics Matrix (Compact 5×6)

This compact matrix provides the small-multiples view we discussed: six columns (strategies) × five rows (metrics). It uses the same metrics table and the integer row-count proxies for storage_size_mb and cu_used.

In [ ]:
# Build and display the 5x6 metrics matrix
import math
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import ticker

try:
    metrics_df = spark.read.table("BenchmarkLakehouse.metrics").toPandas()
except Exception:
    metrics_df = pd.read_csv("metrics.csv")

if 'timestamp' in metrics_df.columns:
    try:
        metrics_df['timestamp'] = pd.to_datetime(metrics_df['timestamp'])
    except Exception:
        pass

cols = [
    ("Warehouse", "Full Refresh", "TC.04.x"),
    ("Warehouse", "Full Compare", "TC.06.x"),
    ("Warehouse", "Incremental", "TC.08.x"),
    ("Delta",     "Full Refresh", "TC.03.x"),
    ("Delta",     "Full Compare", "TC.05.x"),
    ("Delta",     "Incremental", "TC.07.x"),
]
ingest_tc_by_format = {"Delta": "TC.01.x", "Warehouse": "TC.02.x"}

def latest_row_for_testcase(df, tc):
    if tc not in df['test_case_id'].values:
        return None
    sub = df[df['test_case_id'] == tc]
    if 'timestamp' in sub.columns:
        sub = sub.sort_values('timestamp')
    return sub.iloc[-1]

def sget_int(s, colname, default=0):
    if s is None or colname not in s.index:
        return int(default)
    val = s[colname]
    if pd.isna(val):
        return int(default)
    try:
        return int(val)
    except Exception:
        try:
            return int(float(val))
        except Exception:
            return int(default)

def sget_float(s, colname, default=0.0):
    if s is None or colname not in s.index:
        return float(default)
    val = s[colname]
    if pd.isna(val):
        return float(default)
    try:
        return float(val)
    except Exception:
        return float(default)

ingest_rows = []
ingest_time_s = []
update_rows = []
update_time_s = []
storage_rows = []
col_labels = []

for fmt, strat, update_tc in cols:
    col_labels.append(f"{fmt}\n{strat}")
    ingest_tc = ingest_tc_by_format.get(fmt)
    ingest_row = latest_row_for_testcase(metrics_df, ingest_tc)
    ingest_rows.append(sget_int(ingest_row, 'rows', default=0))
    ingest_time_s.append(sget_float(ingest_row, 'ingest_time_s', default=0.0))

    update_row = latest_row_for_testcase(metrics_df, update_tc)
    ur = 0
    if update_row is not None:
        if 'cu_used' in update_row.index and (not pd.isna(update_row['cu_used'])):
            try:
                ur = int(update_row['cu_used'])
            except Exception:
                ur = sget_int(update_row, 'rows', default=0)
        else:
            ur = sget_int(update_row, 'rows', default=0)
    update_rows.append(int(ur))
    update_time_s.append(sget_float(update_row, 'ingest_time_s', default=0.0))

    sr = None
    if update_row is not None and 'storage_size_mb' in update_row.index and (not pd.isna(update_row['storage_size_mb'])):
        try:
            sr = int(update_row['storage_size_mb'])
        except Exception:
            sr = None
    if sr is None and ingest_row is not None and 'storage_size_mb' in ingest_row.index and (not pd.isna(ingest_row['storage_size_mb'])):
        try:
            sr = int(ingest_row['storage_size_mb'])
        except Exception:
            sr = None
    if sr is None:
        sr = 0
    storage_rows.append(int(sr))

matrix_df = pd.DataFrame(
    data=[ingest_rows, ingest_time_s, update_rows, update_time_s, storage_rows],
    index=["Ingestion rows", "Ingestion time_s", "Update rows", "Update time_s", "Storage rows"],
    columns=col_labels
)

print("Metrics matrix (values):")
display(matrix_df)

n_rows, n_cols = matrix_df.shape
fig, axes = plt.subplots(n_rows, n_cols, figsize=(3 * n_cols, 2.2 * n_rows), squeeze=False)
plt.subplots_adjust(hspace=0.6, wspace=0.6, top=0.93)
fig.suptitle("Metrics matrix (rows = metrics, cols = strategies)", fontsize=16)

colors = ["#2a9d8f"] * n_cols

for r_i, metric in enumerate(matrix_df.index):
    vals = matrix_df.loc[metric].astype(float).values
    vmin = 0.0
    vmax = float(np.nanmax(vals)) if len(vals) > 0 else 1.0
    if math.isclose(vmax, 0.0, abs_tol=1e-12):
        vmax = 1.0
    pad = vmax * 0.06
    y_min, y_max = vmin, vmax + pad

    for c_i, col_label in enumerate(matrix_df.columns):
        ax = axes[r_i][c_i]
        val = matrix_df.at[metric, col_label]
        ax.bar([0], [val], width=0.6, color=colors[c_i])
        ax.set_xlim(-0.8, 0.8)
        ax.set_ylim(y_min, y_max)
        ax.set_xticks([])
        ax.yaxis.set_major_locator(ticker.MaxNLocator(3))

        if r_i == 0:
            ax.set_title(col_label, fontsize=9)

        if c_i == 0:
            ax.set_ylabel(metric, fontsize=10)

        try:
            label_text = f"{int(val):,}"
        except Exception:
            label_text = f"{val}"
        ax.text(0, y_min - (y_max - y_min) * 0.14, label_text, ha='center', va='top', fontsize=10)
        for spine in ['top', 'right', 'left']:
            ax.spines[spine].set_visible(False)

plt.show()

summary = matrix_df.T.reset_index().rename(columns={'index': 'strategy'})
print('\nSummary (columns = metrics):')
display(summary)
